
# Classificação de Variedades de Grãos de Trigo (Seeds Dataset) — CRISP-DM
**Resumo:** Notebook que segue a metodologia CRISP-DM para classificar três variedades de grãos de trigo (Kama, Rosa, Canadian) usando o *Seeds Dataset* (UCI). O notebook faz download automático dos dados da UCI, realiza EDA, pré-processamento, treina vários modelos, executa busca de hiperparâmetros e apresenta conclusões interpretáveis.

**Como usar:** execute todas as células (recomendado em Google Colab ou Jupyter local). O dataset será baixado automaticamente da UCI. 


In [ ]:

# Imports e configurações iniciais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [ ]:

# 1) Carregar o dataset direto do UCI Repository
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt'

# O arquivo não contém cabeçalho; colunas conforme descrição UCI
colnames = ['area','perimeter','compactness','length_kernel','width_kernel','asymmetry_coeff','length_groove','class']

df = pd.read_csv(url, sep='\s+', header=None, names=colnames)
print('Shape:', df.shape)
df.head()


In [ ]:

# Estatísticas descritivas
display(df.describe().T)

# Distribuição das classes
display(df['class'].value_counts().sort_index())


In [ ]:

# Visualizações: histogramas e boxplots
features = df.columns[:-1]

plt.figure(figsize=(12,8))
df[features].hist(bins=15, layout=(3,3), figsize=(12,10))
plt.suptitle('Histogramas das características')
plt.show()

plt.figure(figsize=(12,8))
df[features].plot(kind='box', subplots=True, layout=(3,3), figsize=(12,10))
plt.suptitle('Boxplots das características')
plt.show()


In [ ]:

# Scatter matrix (pairplot) para ver relações entre atributos
sns.pairplot(df, vars=features, hue='class', corner=True, plot_kws={'alpha':0.6,'s':30})
plt.suptitle('Pairplot (scatter matrix) por classe', y=1.02)
plt.show()


In [ ]:

# Verificar valores ausentes
print('Missing values por coluna:\n', df.isnull().sum())

# Correlações
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Matriz de correlação')
plt.show()


In [ ]:

# 2) Pré-processamento
X = df.drop('class', axis=1).values
y = df['class'].values

# Treino/teste (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Escalonamento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Dataset split feito. X_train:', X_train.shape, 'X_test:', X_test.shape)


In [ ]:

# Função utilitária para treinar, prever e apresentar métricas
def evaluate_model(model, Xtr, Xte, ytr, yte, verbose=True):
    model.fit(Xtr, ytr)
    ypred = model.predict(Xte)
    acc = accuracy_score(yte, ypred)
    report = classification_report(yte, ypred, zero_division=0)
    cm = confusion_matrix(yte, ypred)
    if verbose:
        print(model.__class__.__name__)
        print('Accuracy: {:.4f}'.format(acc))
        print('Classification report:\n', report)
        print('Confusion matrix:\n', cm)
        print('-'*40)
    return {'model': model, 'accuracy': acc, 'report': report, 'confusion_matrix': cm, 'y_pred': ypred}


In [ ]:

# Modelos iniciais (baseline)
models = [
    KNeighborsClassifier(),
    SVC(probability=True),
    RandomForestClassifier(random_state=42),
    GaussianNB(),
    LogisticRegression(max_iter=1000)
]

results = {}
for m in models:
    res = evaluate_model(m, X_train_scaled, X_test_scaled, y_train, y_test)
    results[m.__class__.__name__] = res


In [ ]:

# Comparar acurácias
for name, r in results.items():
    print(f'{name:20s} -> Accuracy: {r["accuracy"]:.4f}')


In [ ]:

# 3) Otimização de hiperparâmetros (exemplos para KNN, SVM e RandomForest)
param_grids = {
    'KNN': {'n_neighbors': [1,3,5,7,9], 'weights': ['uniform','distance']},
    'SVC': {'C':[0.1,1,10,100], 'gamma': ['scale','auto'], 'kernel':['rbf','poly']},
    'RF': {'n_estimators':[50,100,200], 'max_depth':[None,5,10], 'min_samples_split':[2,5]}
}

# KNN GridSearch
knn = KNeighborsClassifier()
knn_gscv = GridSearchCV(knn, param_grids['KNN'], cv=5, scoring='accuracy', n_jobs=-1)
knn_gscv.fit(X_train_scaled, y_train)
print('Best KNN params:', knn_gscv.best_params_, 'Best CV score:', knn_gscv.best_score_)

# SVC GridSearch (may take longer)
svc = SVC(probability=True)
svc_gscv = GridSearchCV(svc, param_grids['SVC'], cv=5, scoring='accuracy', n_jobs=-1)
svc_gscv.fit(X_train_scaled, y_train)
print('Best SVC params:', svc_gscv.best_params_, 'Best CV score:', svc_gscv.best_score_)

# RandomForest GridSearch
rf = RandomForestClassifier(random_state=42)
rf_gscv = GridSearchCV(rf, param_grids['RF'], cv=5, scoring='accuracy', n_jobs=-1)
rf_gscv.fit(X_train_scaled, y_train)
print('Best RF params:', rf_gscv.best_params_, 'Best CV score:', rf_gscv.best_score_)


In [ ]:

# Avaliar modelos otimizados no conjunto de teste
best_models = {
    'KNN': knn_gscv.best_estimator_,
    'SVC': svc_gscv.best_estimator_,
    'RF': rf_gscv.best_estimator_
}

tuned_results = {}
for name, bm in best_models.items():
    tuned_results[name] = evaluate_model(bm, X_train_scaled, X_test_scaled, y_train, y_test)


In [ ]:

# Comparação de desempenho (baseline -> otimizado)
print('\nComparação de desempenho (baseline -> otimizado):')
for cls_name in ['KNeighborsClassifier','SVC','RandomForestClassifier']:
    # Map baseline class names to tuned_results keys
    key = 'KNN' if 'KNeighbors' in cls_name else ('SVC' if 'SVC' in cls_name else 'RF')
    base_acc = results.get(cls_name, {}).get('accuracy', 0)
    tuned_acc = tuned_results.get(key, {}).get('accuracy', 0)
    print(f'{cls_name:25s} -> baseline: {base_acc:.4f}  | tuned: {tuned_acc:.4f}')


In [ ]:

# Importância de features (se RandomForest estiver disponível)
if 'RF' in best_models:
    rf_best = best_models['RF']
    importances = rf_best.feature_importances_
    feat_imp = pd.Series(importances, index=features).sort_values(ascending=False)
    print('Feature importances (RandomForest):')
    display(feat_imp)
    plt.figure(figsize=(8,4))
    feat_imp.plot(kind='bar')
    plt.title('Importância das features (RandomForest)')
    plt.tight_layout()
    plt.show()



## Conclusões e insights
- Resuma aqui o desempenho dos modelos, quais features foram mais importantes e se a classificação é robusta para aplicação em cooperativas agrícolas de pequeno porte.
- Discuta limitações (por exemplo: tamanho do dataset, necessidade de imagens originais para melhorar via deep learning, variabilidade entre lotes) e próximos passos (coleta de mais dados, validação em campo, pipeline de produção para inferência em tempo real).
